In [1]:
import os
import tarfile
import urllib
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedShuffleSplit

In [2]:
DOWNLOAD_ROOT = "https://raw.githubusercontent.com/ageron/handson-ml2/master/"
HOUSING_PATH = os.path.join("datasets", "housing")
HOUSING_URL = DOWNLOAD_ROOT + "datasets/housing/housing.tgz"

def fetch_housing_data(housing_url=HOUSING_URL, housing_path=HOUSING_PATH):
    os.makedirs(housing_path, exist_ok=True)
    tgz_path = os.path.join(housing_path, "housing.tgz")
    urllib.request.urlretrieve(housing_url, tgz_path)
    housing_tgz = tarfile.open(tgz_path)
    housing_tgz.extractall(path=housing_path)
    housing_tgz.close()

In [3]:
def load_housing_data(housing_path=HOUSING_PATH):
    csv_path = os.path.join(housing_path, "housing.csv")
    return pd.read_csv(csv_path)

In [4]:
fetch_housing_data()
housing = load_housing_data()

In [5]:
housing.head()

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value,ocean_proximity
0,-122.23,37.88,41.0,880.0,129.0,322.0,126.0,8.3252,452600.0,NEAR BAY
1,-122.22,37.86,21.0,7099.0,1106.0,2401.0,1138.0,8.3014,358500.0,NEAR BAY
2,-122.24,37.85,52.0,1467.0,190.0,496.0,177.0,7.2574,352100.0,NEAR BAY
3,-122.25,37.85,52.0,1274.0,235.0,558.0,219.0,5.6431,341300.0,NEAR BAY
4,-122.25,37.85,52.0,1627.0,280.0,565.0,259.0,3.8462,342200.0,NEAR BAY


In [6]:
housing["income_cat"] = pd.cut(housing["median_income"],
                               bins=[0., 1.5, 3.0, 4.5, 6., np.inf],
                               labels=[1, 2, 3, 4, 5])

In [7]:
split = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=42)
for train_index, test_index in split.split(housing, housing["income_cat"]):
    strat_train_set = housing.loc[train_index]
    strat_test_set = housing.loc[test_index]

In [8]:
for set_ in (strat_train_set, strat_test_set):
    set_.drop("income_cat", axis=1, inplace=True)

In [9]:
housing = strat_train_set.copy()

In [10]:
housing["rooms_per_household"] = housing["total_rooms"]/housing["households"]
housing["bedrooms_per_room"] = housing["total_bedrooms"]/housing["total_rooms"]
housing["population_per_household"]=housing["population"]/housing["households"]

In [11]:
housing = strat_train_set.drop("median_house_value", axis=1)
housing_labels = strat_train_set["median_house_value"].copy()

In [12]:
from sklearn.impute import SimpleImputer

imputer = SimpleImputer(strategy="median")

In [13]:
housing_num = housing.drop("ocean_proximity", axis=1)

In [14]:
imputer.fit(housing_num)

SimpleImputer(strategy='median')

In [15]:
X = imputer.transform(housing_num)

In [16]:
X

array([[-1.2146e+02,  3.8520e+01,  2.9000e+01, ...,  2.2370e+03,
         7.0600e+02,  2.1736e+00],
       [-1.1723e+02,  3.3090e+01,  7.0000e+00, ...,  2.0150e+03,
         7.6800e+02,  6.3373e+00],
       [-1.1904e+02,  3.5370e+01,  4.4000e+01, ...,  6.6700e+02,
         3.0000e+02,  2.8750e+00],
       ...,
       [-1.2272e+02,  3.8440e+01,  4.8000e+01, ...,  4.5800e+02,
         1.7200e+02,  3.1797e+00],
       [-1.2270e+02,  3.8310e+01,  1.4000e+01, ...,  1.2080e+03,
         5.0100e+02,  4.1964e+00],
       [-1.2214e+02,  3.9970e+01,  2.7000e+01, ...,  6.2500e+02,
         1.9700e+02,  3.1319e+00]])

In [17]:
housing_tr = pd.DataFrame(X, columns=housing_num.columns,
                          index=housing_num.index)

In [18]:
housing_tr

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income
12655,-121.46,38.52,29.0,3873.0,797.0,2237.0,706.0,2.1736
15502,-117.23,33.09,7.0,5320.0,855.0,2015.0,768.0,6.3373
2908,-119.04,35.37,44.0,1618.0,310.0,667.0,300.0,2.8750
14053,-117.13,32.75,24.0,1877.0,519.0,898.0,483.0,2.2264
20496,-118.70,34.28,27.0,3536.0,646.0,1837.0,580.0,4.4964
...,...,...,...,...,...,...,...,...
15174,-117.07,33.03,14.0,6665.0,1231.0,2026.0,1001.0,5.0900
12661,-121.42,38.51,15.0,7901.0,1422.0,4769.0,1418.0,2.8139
19263,-122.72,38.44,48.0,707.0,166.0,458.0,172.0,3.1797
19140,-122.70,38.31,14.0,3155.0,580.0,1208.0,501.0,4.1964


In [19]:
housing_cat = housing[["ocean_proximity"]]

In [20]:
from sklearn.preprocessing import OrdinalEncoder
ordinal_encoder = OrdinalEncoder()

In [21]:
housing_cat_encoded = ordinal_encoder.fit_transform(housing_cat)
housing_cat_encoded[:10]

array([[1.],
       [4.],
       [1.],
       [4.],
       [0.],
       [3.],
       [0.],
       [0.],
       [0.],
       [0.]])

In [22]:
from sklearn.preprocessing import OneHotEncoder
cat_encoder = OneHotEncoder()

In [23]:
housing_cat_1hot = cat_encoder.fit_transform(housing_cat)
housing_cat_1hot

<16512x5 sparse matrix of type '<class 'numpy.float64'>'
	with 16512 stored elements in Compressed Sparse Row format>

In [24]:
housing_cat_1hot.toarray()

array([[0., 1., 0., 0., 0.],
       [0., 0., 0., 0., 1.],
       [0., 1., 0., 0., 0.],
       ...,
       [1., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0.]])

In [25]:
from sklearn.base import BaseEstimator, TransformerMixin

rooms_ix, bedrooms_ix, population_ix, households_ix = 3, 4, 5, 6

class CombinedAttributesAdder(BaseEstimator, TransformerMixin):
    def __init__(self, add_bedrooms_per_room = True): # no *args or **kargs
        self.add_bedrooms_per_room = add_bedrooms_per_room
    def fit(self, X, y=None):
        return self  # nothing else to do
    def transform(self, X):
        rooms_per_household = X[:, rooms_ix] / X[:, households_ix]
        population_per_household = X[:, population_ix] / X[:, households_ix]
        if self.add_bedrooms_per_room:
            bedrooms_per_room = X[:, bedrooms_ix] / X[:, rooms_ix]
            return np.c_[X, rooms_per_household, population_per_household,
                         bedrooms_per_room]

        else:
            return np.c_[X, rooms_per_household, population_per_household]

attr_adder = CombinedAttributesAdder(add_bedrooms_per_room=False)
housing_extra_attribs = attr_adder.transform(housing.values)

In [27]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler

num_pipeline = Pipeline([
        ('imputer', SimpleImputer(strategy="median")),
        ('attribs_adder', CombinedAttributesAdder()),
        ('std_scaler', StandardScaler()),
    ])

housing_num_tr = num_pipeline.fit_transform(housing_num)

In [28]:
from sklearn.compose import ColumnTransformer

num_attribs = list(housing_num.columns)
cat_attribs = ["ocean_proximity"]

full_pipeline = ColumnTransformer([
        ("num", num_pipeline, num_attribs),
        ("cat", OneHotEncoder(), cat_attribs),
    ])

housing_prepared = full_pipeline.fit_transform(housing)

## Exercise 1 -> Try a Support Vector Machine regressor (sklearn.svm.SVR) with various hyperparameters, such as kernel="linear" (with various values for the C hyperparameter) or kernel="rbf" (with various values for the C and gamma hyperparameters).

In [29]:
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVR
from sklearn.metrics import mean_squared_error

# Define the parameter grid for grid search
param_grid = [
    {
        'kernel': ['linear'],
        'C': [0.1, 1, 10, 100]
    },
    {
        'kernel': ['rbf'],
        'C': [0.1, 1, 10, 100],
        'gamma': [0.01, 0.1, 1, 10]
    }
]

# Create the SVR model
svr = SVR()

# Perform grid search with cross-validation
grid_search = GridSearchCV(svr, param_grid, cv=5, scoring='neg_mean_squared_error')
grid_search.fit(housing_prepared, housing_labels)

# Get the best parameters and best score
best_params = grid_search.best_params_
best_score = -grid_search.best_score_  # converting from negative MSE to MSE

# Print the best parameters and the best score
print("Best Parameters:", best_params)
print("Best Cross-Validation MSE:", best_score)

# Evaluate the best model on the test set


best_model = grid_search.best_estimator_
X_test = strat_test_set.drop("median_house_value", axis=1)
y_test = strat_test_set["median_house_value"].copy()

X_test_prepared = full_pipeline.transform(X_test)

y_pred = best_model.predict(X_test_prepared)
final_mse = mean_squared_error(y_test, y_pred)
final_rmse = np.sqrt(final_mse)

print("Test Set MSE:", final_rmse)


Best Parameters: {'C': 100, 'kernel': 'linear'}
Best Cross-Validation MSE: 5131652543.609375
Test Set MSE: 69499.58221771519


## Exercise 2 -> Try replacing GridSearchCV with RandomizedSearchCV.

In [39]:
from scipy.stats import uniform
from sklearn.model_selection import RandomizedSearchCV
# Define the parameter distribution for random search
param_distributions = {
    'kernel': ['linear', 'rbf'],
    'C': uniform(0.1, 100),  # Continuous uniform distribution over [0.1, 100)
    'gamma': uniform(0.01, 10)  # Continuous uniform distribution over [0.01, 10) for the rbf kernel
}

# Create the SVR model
svr = SVR()

# Perform random search with cross-validation
random_search = RandomizedSearchCV(svr, param_distributions, n_iter=50, cv=5, scoring='neg_mean_squared_error', random_state=42)
random_search.fit(housing_prepared, housing_labels)

# Get the best parameters and best score
best_params = random_search.best_params_
best_score = -random_search.best_score_  # converting from negative MSE to MSE

# Print the best parameters and the best score
print("Best Parameters:", best_params)
print("Best Cross-Validation MSE:", best_score)

# Evaluate the best model on the test set


best_model = random_search.best_estimator_
X_test = strat_test_set.drop("median_house_value", axis=1)
y_test = strat_test_set["median_house_value"].copy()

X_test_prepared = full_pipeline.transform(X_test)

y_pred = best_model.predict(X_test_prepared)
final_mse = mean_squared_error(y_test, y_pred)
final_rmse = np.sqrt(final_mse)

print("Test Set MSE:", final_rmse)

Best Parameters: {'C': 98.42308858067881, 'gamma': 4.677628932479799, 'kernel': 'linear'}
Best Cross-Validation MSE: 5136122888.131926
Test Set MSE: 69526.16775916264


## Exercise 3 -> Try adding a transformer in the preparation pipeline to select only the most important attributes.

In [48]:
def indices_of_top_k(arr, k):
    return np.argsort(arr)[-k:]

In [51]:
from sklearn.base import BaseEstimator, TransformerMixin

class SelectTopFeatures(BaseEstimator, TransformerMixin):
    def __init__(self, feature_importances, k):
        self.feature_importances = feature_importances
        self.k = k
    def fit(self, X, y=None):
        self.feature_indices_ = indices_of_top_k(self.feature_importances, self.k)
        return self
    def transform(self, X):
        return X[:, self.feature_indices_]

In [44]:
# Using Random Forest Regressor inorder to get the top 5
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestRegressor
param_grid = [
    {'n_estimators': [3, 10, 30], 'max_features': [2, 4, 6, 8]},
    {'bootstrap': [False], 'n_estimators': [3, 10], 'max_features': [2, 3, 4]},
  ]

forest_reg = RandomForestRegressor()

grid_search = GridSearchCV(forest_reg, param_grid, cv=5,
                           scoring='neg_mean_squared_error',
                           return_train_score=True)

grid_search.fit(housing_prepared, housing_labels)

GridSearchCV(cv=5, estimator=RandomForestRegressor(),
             param_grid=[{'max_features': [2, 4, 6, 8],
                          'n_estimators': [3, 10, 30]},
                         {'bootstrap': [False], 'max_features': [2, 3, 4],
                          'n_estimators': [3, 10]}],
             return_train_score=True, scoring='neg_mean_squared_error')

In [45]:
feature_importances = grid_search.best_estimator_.feature_importances_
feature_importances

array([7.56051003e-02, 6.72226895e-02, 4.31377164e-02, 1.73520027e-02,
       1.70520211e-02, 1.67450443e-02, 1.61758668e-02, 3.48263858e-01,
       5.95323422e-02, 1.09315747e-01, 6.22981726e-02, 1.88571860e-02,
       1.40194514e-01, 6.50889648e-05, 2.47914923e-03, 5.70350085e-03])

In [46]:
k = 5
top_k_feature_indices = indices_of_top_k(feature_importances, k)
top_k_feature_indices

array([ 0,  1,  7,  9, 12])

In [49]:
attributes = ['longitude', 'latitude', 'housing_median_age', 'total_rooms', 'total_bedrooms', 'population', 'households', 'median_income', 'rooms_per_hhold', 'pop_per_hhold', 'bedrooms_per_room', '<1H OCEAN', 'INLAND', 'ISLAND', 'NEAR BAY', 'NEAR OCEAN']
np.array(attributes)[top_k_feature_indices]

array(['longitude', 'latitude', 'median_income', 'pop_per_hhold',
       'INLAND'], dtype='<U18')

In [52]:
preparation_and_feature_selection_pipeline = Pipeline([
    ('preparation', full_pipeline),
    ('feature_selection', SelectTopFeatures(feature_importances, k))
])

In [53]:
housing_prepared_top_k_features = preparation_and_feature_selection_pipeline.fit_transform(housing)

In [54]:
# The below 5 features can be used for the creation of the model
housing_prepared_top_k_features[0]

array([ 1.34743822, -0.94135046,  0.00622264,  1.        , -0.8936472 ])

## Exercise 4 -> Try creating a single pipeline that does the full data preparation plus the final prediction.

In [55]:
prepare_select_and_predict_pipeline = Pipeline([
    ('preparation', full_pipeline),
    ('feature_selection', SelectTopFeatures(feature_importances, k)),
    ('svm_reg', SVR(**random_search.best_params_))
])

In [56]:
prepare_select_and_predict_pipeline.fit(housing, housing_labels)

Pipeline(steps=[('preparation',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='median')),
                                                                  ('attribs_adder',
                                                                   CombinedAttributesAdder()),
                                                                  ('std_scaler',
                                                                   StandardScaler())]),
                                                  ['longitude', 'latitude',
                                                   'housing_median_age',
                                                   'total_rooms',
                                                   'total_bedrooms',
                                                   'population', 'households',
                                                   'median_income']),
                                                 ('cat', OneHotEncoder(...
                 SelectTopFeatures(feature_importances=array([7.56051003e-02, 6.72226895e-02, 4.31377164e-02, 1.73520027e-02,
       1.70520211e-02, 1.67450443e-02, 1.61758668e-02, 3.48263858e-01,
       5.95323422e-02, 1.09315747e-01, 6.22981726e-02, 1.88571860e-02,
       1.40194514e-01, 6.50889648e-05, 2.47914923e-03, 5.70350085e-03]),
                                   k=5)),
                ('svm_reg',
                 SVR(C=98.42308858067881, gamma=4.677628932479799,
                     kernel='linear'))])

In [57]:
print(prepare_select_and_predict_pipeline.predict(housing))
print(list(housing_labels))

Predictions:	 [ 87757.87124711 291722.50017721 116157.00298262 ... 193550.90117706
 231227.90209032 114676.82616633]
Labels:		 [72100.0, 279600.0, 82700.0, 112500.0, 238300.0, 165700.0, 412300.0, 349600.0, 181300.0, 500001.0, 57500.0, 350000.0, 238800.0, 80300.0, 200400.0, 333700.0, 158000.0, 276800.0, 327200.0, 194500.0, 163100.0, 331200.0, 92000.0, 153400.0, 139100.0, 85900.0, 238100.0, 173500.0, 196100.0, 442900.0, 234600.0, 195400.0, 264100.0, 415800.0, 282300.0, 345200.0, 85000.0, 279400.0, 95700.0, 191800.0, 225400.0, 163500.0, 150300.0, 217100.0, 157500.0, 163200.0, 105100.0, 213700.0, 98200.0, 235300.0, 182400.0, 390800.0, 179900.0, 46700.0, 339800.0, 151400.0, 105000.0, 183300.0, 125700.0, 123600.0, 249400.0, 152600.0, 293300.0, 116700.0, 133000.0, 261900.0, 87500.0, 126500.0, 261600.0, 225800.0, 196500.0, 109800.0, 174400.0, 217700.0, 172400.0, 325000.0, 132200.0, 184800.0, 163700.0, 142500.0, 50000.0, 307400.0, 253700.0, 340400.0, 72300.0, 121800.0, 85000.0, 148200.0, 118800

## 5. Automatically explore some preparation options using GridSearchCV.

In [58]:
full_pipeline.named_transformers_["cat"].handle_unknown = 'ignore'

param_grid = [{
    'preparation__num__imputer__strategy': ['mean', 'median', 'most_frequent'],
    'feature_selection__k': list(range(1, len(feature_importances) + 1))
}]

grid_search_prep = GridSearchCV(prepare_select_and_predict_pipeline, param_grid, cv=5,
                                scoring='neg_mean_squared_error', verbose=2)
grid_search_prep.fit(housing, housing_labels)

Fitting 5 folds for each of 48 candidates, totalling 240 fits


/home/prithiveer/mle-training/venv_jupyter/lib/python3.12/site-packages/sklearn/model_selection/_validation.py:982: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/home/prithiveer/mle-training/venv_jupyter/lib/python3.12/site-packages/sklearn/model_selection/_validation.py", line 971, in _score
    scores = scorer(estimator, X_test, y_test, **score_params)
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/prithiveer/mle-training/venv_jupyter/lib/python3.12/site-packages/sklearn/metrics/_scorer.py", line 279, in __call__
    return self._score(partial(_cached_call, None), estimator, X, y_true, **_kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/prithiveer/mle-training/venv_jupyter/lib/python3.12/site-packages/sklearn/metrics/_scorer.py", line 371, in _score
    y_pred = method_caller(
     

[CV] END feature_selection__k=1, preparation__num__imputer__strategy=mean; total time=   4.8s
[CV] END feature_selection__k=1, preparation__num__imputer__strategy=mean; total time=   4.7s
[CV] END feature_selection__k=1, preparation__num__imputer__strategy=mean; total time=   4.9s
[CV] END feature_selection__k=1, preparation__num__imputer__strategy=mean; total time=   4.6s
[CV] END feature_selection__k=1, preparation__num__imputer__strategy=mean; total time=   5.1s


/home/prithiveer/mle-training/venv_jupyter/lib/python3.12/site-packages/sklearn/model_selection/_validation.py:982: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/home/prithiveer/mle-training/venv_jupyter/lib/python3.12/site-packages/sklearn/model_selection/_validation.py", line 971, in _score
    scores = scorer(estimator, X_test, y_test, **score_params)
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/prithiveer/mle-training/venv_jupyter/lib/python3.12/site-packages/sklearn/metrics/_scorer.py", line 279, in __call__
    return self._score(partial(_cached_call, None), estimator, X, y_true, **_kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/prithiveer/mle-training/venv_jupyter/lib/python3.12/site-packages/sklearn/metrics/_scorer.py", line 371, in _score
    y_pred = method_caller(
     

[CV] END feature_selection__k=1, preparation__num__imputer__strategy=median; total time=   4.3s
[CV] END feature_selection__k=1, preparation__num__imputer__strategy=median; total time=   4.8s
[CV] END feature_selection__k=1, preparation__num__imputer__strategy=median; total time=   4.7s
[CV] END feature_selection__k=1, preparation__num__imputer__strategy=median; total time=   4.5s
[CV] END feature_selection__k=1, preparation__num__imputer__strategy=median; total time=   4.6s


...

[CV] END feature_selection__k=13, preparation__num__imputer__strategy=median; total time=   5.5s
[CV] END feature_selection__k=13, preparation__num__imputer__strategy=median; total time=   6.0s
[CV] END feature_selection__k=13, preparation__num__imputer__strategy=median; total time=   6.7s
[CV] END feature_selection__k=13, preparation__num__imputer__strategy=median; total time=   6.2s
[CV] END feature_selection__k=13, preparation__num__imputer__strategy=median; total time=   6.4s


/home/prithiveer/mle-training/venv_jupyter/lib/python3.12/site-packages/sklearn/model_selection/_validation.py:982: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/home/prithiveer/mle-training/venv_jupyter/lib/python3.12/site-packages/sklearn/model_selection/_validation.py", line 971, in _score
    scores = scorer(estimator, X_test, y_test, **score_params)
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/prithiveer/mle-training/venv_jupyter/lib/python3.12/site-packages/sklearn/metrics/_scorer.py", line 279, in __call__
    return self._score(partial(_cached_call, None), estimator, X, y_true, **_kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/prithiveer/mle-training/venv_jupyter/lib/python3.12/site-packages/sklearn/metrics/_scorer.py", line 371, in _score
    y_pred = method_caller(
     

[CV] END feature_selection__k=13, preparation__num__imputer__strategy=most_frequent; total time=   5.5s
[CV] END feature_selection__k=13, preparation__num__imputer__strategy=most_frequent; total time=   6.4s
[CV] END feature_selection__k=13, preparation__num__imputer__strategy=most_frequent; total time=   6.2s
[CV] END feature_selection__k=13, preparation__num__imputer__strategy=most_frequent; total time=   6.4s
[CV] END feature_selection__k=13, preparation__num__imputer__strategy=most_frequent; total time=   6.5s
[CV] END feature_selection__k=14, preparation__num__imputer__strategy=mean; total time=   0.0s
[CV] END feature_selection__k=14, preparation__num__imputer__strategy=mean; total time=   7.0s
[CV] END feature_selection__k=14, preparation__num__imputer__strategy=mean; total time=   6.8s
[CV] END feature_selection__k=14, preparation__num__imputer__strategy=mean; total time=   6.3s
[CV] END feature_selection__k=14, preparation__num__imputer__strategy=mean; total time=   6.4s
[CV] 

/home/prithiveer/mle-training/venv_jupyter/lib/python3.12/site-packages/sklearn/model_selection/_validation.py:540: FitFailedWarning: 
9 fits failed out of a total of 240.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
9 fits failed with the following error:
Traceback (most recent call last):
  File "/home/prithiveer/mle-training/venv_jupyter/lib/python3.12/site-packages/sklearn/model_selection/_validation.py", line 888, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/home/prithiveer/mle-training/venv_jupyter/lib/python3.12/site-packages/sklearn/base.py", line 1473, in wrapper
    return fit_method(estimator, *args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/prithiveer/mle-trai

GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('preparation',
                                        ColumnTransformer(transformers=[('num',
                                                                         Pipeline(steps=[('imputer',
                                                                                          SimpleImputer(strategy='median')),
                                                                                         ('attribs_adder',
                                                                                          CombinedAttributesAdder()),
                                                                                         ('std_scaler',
                                                                                          StandardScaler())]),
                                                                         ['longitude',
                                                                          'latitude',
                                                                          'housing_median_age',
                                                                          'total_rooms',
                                                                          'total_bedrooms',
                                                                          'population',
                                                                          'households',
                                                                          'median_inc...
       5.95323422e-02, 1.09315747e-01, 6.22981726e-02, 1.88571860e-02,
       1.40194514e-01, 6.50889648e-05, 2.47914923e-03, 5.70350085e-03]),
                                                          k=5)),
                                       ('svm_reg',
                                        SVR(C=98.42308858067881,
                                            gamma=4.677628932479799,
                                            kernel='linear'))]),
             param_grid=[{'feature_selection__k': [1, 2, 3, 4, 5, 6, 7, 8, 9,
                                                   10, 11, 12, 13, 14, 15, 16],
                          'preparation__num__imputer__strategy': ['mean',
                                                                  'median',
                                                                  'most_frequent']}],
             scoring='neg_mean_squared_error', verbose=2)

In [59]:
grid_search_prep.best_params_

{'feature_selection__k': 1, 'preparation__num__imputer__strategy': 'mean'}